# Import data

In [ ]:
# Import libraries
import os
import requests
import boto3
from botocore.exceptions import NoCredentialsError
import pyarrow.parquet as pq
import pandas as pd
from tqdm import tqdm

In [ ]:
# Fonction de téléchargement des données depuis S3 ou une URL avec barre de progression
def download_data(local_path, url=None, s3_path=None, bucket=None, endpoint=None):
    """
    Télécharge un fichier depuis S3 ou une URL et le sauvegarde localement.
    """
    # Créer le dossier si nécessaire
    dir_path = os.path.dirname(local_path)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Vérifier si le fichier existe déjà
    if os.path.exists(local_path):
        print(f"Les données sont déjà téléchargées : {local_path}")
        return

    # Télécharger depuis S3
    if s3_path and bucket and endpoint:
        print("Téléchargement depuis le bucket S3...")
        try:
            s3 = boto3.client('s3', endpoint_url=endpoint)
            s3.download_file(bucket, s3_path, local_path)
            print(f"Téléchargement effectué avec succès depuis S3 : {local_path}")
        except NoCredentialsError as e:
            print(f"Erreur d'authentification S3 : {e}")
        except Exception as e:
            print(f"Erreur lors du téléchargement depuis S3 : {e}")
            return

    # Télécharger depuis une URL
    elif url:
        print("Téléchargement depuis l'URL...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Barre de progression
            total_size = int(response.headers.get('content-length', 0))
            with open(local_path, 'wb') as f, tqdm(
                desc=f"Téléchargement {os.path.basename(local_path)}",
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    f.write(data)
                    bar.update(len(data))
            print(f"Téléchargement effectué avec succès : {local_path}")
        except requests.RequestException as e:
            print(f"Erreur lors du téléchargement depuis l'URL : {e}")
            return

    else:
        print("Ni URL, ni chemin S3 spécifié. Impossible de télécharger les données.")
        return

# Fonction pour lire les fichiers Parquet
def read_parquet_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pq.read_table(local_path).to_pandas()
            print(f"Données Parquet chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier Parquet : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

# Fonction pour lire les fichiers CSV
def read_csv_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pd.read_csv(local_path, sep=';')
            print(f"Données CSV chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier CSV : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

In [ ]:
# Définir le répertoire courant
os.chdir('/home/onyxia/work/methodes_ensemblistes_notebooks')

# Vérification
print("Nouveau répertoire courant :", os.getcwd())

In [ ]:
# Téléchargement des fichiers en local

# Depuis une URL
download_data(
    local_path="data/data_census_individuals.parquet",
    url="https://www.data.gouv.fr/fr/datasets/r/c8e1b241-75fe-43e9-a266-830fc30ec61d"
)
download_data(
    local_path="data/data_census_dwellings.parquet",
    url="https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2"
)
download_data(
    local_path="documentation/doc_census_individuals.csv",
    url="https://www.data.gouv.fr/fr/datasets/r/1c6c6ab2-b766-41a4-90f0-043173d5e9d1"
)
download_data(
    local_path="documentation/doc_census_dwellings.csv",
    url="https://www.data.gouv.fr/fr/datasets/r/c274705f-98db-4d9b-9674-578e04f03198"
)

In [ ]:
# Lecture des fichiers Parquet
data_census_individuals = read_parquet_data("data/data_census_individuals.parquet")
data_census_dwellings = read_parquet_data("data/data_census_dwellings.parquet")

# Lecture des fichiers CSV
doc_census_individuals = read_csv_data("documentation/doc_census_individuals.csv")
doc_census_dwellings = read_csv_data("documentation/doc_census_dwellings.csv")

In [ ]:
# Afficher un aperçu des données
print(data_census_individuals.head())
print(data_census_dwellings.head())
print(doc_census_individuals.head())
print(doc_census_dwellings.head())

In [ ]:
doc_census_individuals = read_csv_data("documentation/doc_census_individuals.csv")
print("doc_census_individuals" in globals()) 